In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from collections import Counter, defaultdict
import tqdm
import json
import codecs

In [2]:
# файл с информацией о документах

doc = pd.read_csv('annotator_document_11.12.csv', sep='^')

In [3]:
# файл с предложениями

def open_csv_sent(file):
    f = open(file, 'r', encoding = 'utf-8')
    s = []
    l = f.readlines()
    f.close()
    for i in l:
        i = i.replace('"\n', '')
        if 'NULL' in i:
            i = i.replace('NULL', '"NULL"')
        s.append(i[1:].split('"#"'))
    return s

In [4]:
# файл с ошибками

def open_csv_errors(file):
    f = open(file, 'r', encoding = 'utf-8') 
    s = []
    l = f.readlines()
    f.close()
    for i in l:
        i = i.replace('"\n', '')
        i = i.replace('##', '#""#')
        i = i.replace('#\n', '#"')
        if 'NULL' in i:
            i = i.replace('NULL', '"NULL"')
        istr = i[1:].split('"#"')
        if len(istr) == 13:
            istr.append('')
        s.append(istr)
    return s

In [5]:
sent = open_csv_sent('annotator_sentence_11.12.csv')
annot = open_csv_errors('annotator_annotation_18.12.csv')

In [6]:
doc_fr = doc.loc[doc['native'] == 'fr',]

In [222]:
#doc_an = doc_fr.loc[doc_fr['annotated'] == 1.0,]  # часть размеченных данных не помечена как размеченные
doc_an = doc_fr
doc_fl = doc_an.loc[doc_an['language_background'] == 'FL']
doc_hl = doc_an.loc[doc_an['language_background'] == 'HL']

In [223]:
# достаем нужные предложения

def get_sent():
    d_id_fl = list(doc_fl['id']) 
    d_id_hl = list(doc_hl['id']) 
    
    sent_fl = pd.DataFrame(columns = sent[0])
    sent_hl = pd.DataFrame(columns = sent[0])
    k = 0
    m = 0
    for s in tqdm.tqdm_notebook(sent):
        try:
            if int(s[2]) in d_id_fl:
                sent_fl.loc[k] = s
                k += 1
            elif int(s[2]) in d_id_hl:
                sent_hl.loc[m] = s
                m += 1
        except:
            print(s)
            
    return sent_fl, sent_hl

In [224]:
sent_fl, sent_hl = get_sent()

['id', 'text', 'doc_id_id', 'num', 'tagged', 'correct', 'temp', 'correct2']
['107393"##"6339', '1"###"NULL"#\n']



In [225]:
# достаем нужные ошибки

def get_errors():
    sent_id_fl = list(sent_fl['id'])
    sent_id_hl = list(sent_hl['id'])
    
    errors_fl = pd.DataFrame(columns = annot[0])
    errors_hl = pd.DataFrame(columns = annot[0])
    k = 0
    m = 0
    for e in tqdm.tqdm_notebook(annot):
        try:
            if e[2] in sent_id_fl:
                errors_fl.loc[k] = e
                k += 1
            if e[2] in sent_id_hl:
                errors_hl.loc[m] = e
                m += 1
        except IndexError:
            print(e)
            
    return errors_fl, errors_hl

In [226]:
errors_fl, errors_hl = get_errors()

## Количество текстов

In [227]:
doc_fl['level'].value_counts()

A2    159
A1     71
B1     59
B2     27
C1      9
Name: level, dtype: int64

In [228]:
doc_hl['level'].value_counts()

C2    62
C1    36
B2     8
B1     6
Name: level, dtype: int64

**Смотрим какие есть теги. Lex и Morph не будем брать, т.к. это ошибки разметки и их мало.**

In [229]:
def get_tags():
    tags_fl = list(errors_fl['tag'])
    tags_hl = list(errors_hl['tag'])
    prep_fl = defaultdict(int)
    prep_hl = defaultdict(int)
    
    for t in tags_fl:
        if 'Prep' in t:
            prep_fl[t] += 1
            
    for t in tags_hl:
        if 'Prep' in t:
            prep_hl[t] += 1


    return prep_fl, prep_hl

In [230]:
prep_fl, prep_hl = get_tags()

In [231]:
prep_fl

defaultdict(int,
            {'Lex, Const, Prep, Miss, Transfer': 1,
             'Lex, Constr, Prep, Extra': 1,
             'Lex, Constr, Prep, Subst': 9,
             'Lex, Constr, Prep, Subst, Fusion': 2,
             'Lex, Constr, Prep, Subst, Transfer': 1,
             'Syntax, Constr, Prep': 12,
             'Syntax, Constr, Prep, Extra': 2,
             'Syntax, Constr, Prep, Extra, Fusion': 2,
             'Syntax, Constr, Prep, Extra, Transfer': 16,
             'Syntax, Constr, Prep, Fusion': 23,
             'Syntax, Constr, Prep, Miss': 18,
             'Syntax, Constr, Prep, Miss, Transfer': 12,
             'Syntax, Constr, Prep, Subst': 77,
             'Syntax, Constr, Prep, Subst, Fusion': 2,
             'Syntax, Constr, Prep, Subst, Transfer': 16,
             'Syntax, Constr, Prep, Transfer': 8,
             'Syntax, Gov, Prep': 32,
             'Syntax, Gov, Prep, Extra': 53,
             'Syntax, Gov, Prep, Extra, Fusion': 10,
             'Syntax, Gov, Prep, Ext

In [232]:
prep_hl

defaultdict(int,
            {'Lex, Constr, Prep, Subst': 2,
             'Lex, Constr, Prep, Subst, Fusion': 1,
             'Lex, Prep': 1,
             'Morph, Altern, Prep': 1,
             'Syntax, Const, Prep, Transfer': 1,
             'Syntax, Constr, Prep': 2,
             'Syntax, Constr, Prep, Miss': 4,
             'Syntax, Constr, Prep, Miss, Transfer': 2,
             'Syntax, Constr, Prep, Subst': 14,
             'Syntax, Constr, Prep, Subst, Fusion': 3,
             'Syntax, Constr, Prep, Transfer': 1,
             'Syntax, Gov, Prep': 1,
             'Syntax, Gov, Prep, Extra': 7,
             'Syntax, Gov, Prep, Extra, Fusion': 2,
             'Syntax, Gov, Prep, Extra, Transfer': 1,
             'Syntax, Gov, Prep, Miss': 16,
             'Syntax, Gov, Prep, Miss, Fusion': 1,
             'Syntax, Gov, Prep, Miss, Transfer': 3,
             'Syntax, Gov, Prep, Subst': 3,
             'Syntax, Gov, Prep, Subst, Fusion': 1,
             'Syntax, Gov, Prep, Subst, Tran

In [258]:
# таблица с информацией по тегам

def take_info(df, doc, sents, errors):
    
    tags = list(errors['tag'])
    
    new_data = []
    for i in range(len(errors)):
        try:
            if 'Prep' in tags[i] and 'Lex' not in tags[i] and 'Morph' not in tags[i]:
                data = errors.data[i].replace('\\", ', '" , ')
                data = data.replace('\\\\', '\\')
                data = json.loads(codecs.decode(data, 'unicode_escape'))
                
                t = tags[i].split(',')
                gc = t[1]
                ems = t[3] if len(t) > 3 else '-'    #transfer
                ft = t[4] if len(t) == 5 else '-'
                
                sf = sents.loc[sents['id'] == errors['document_id'][i],]
                l = doc.loc[doc_an['id'] == int(sf['doc_id_id']),]
                level = list(l['level'])[0]
                
                sentence = list(sf['text'])[0]
                
                before = data['text']
                
                calque = 0
                ind_fr = data['text'].find('фр')
                if ind_fr != -1:
                    calque = 1
                    data['text'] = data['text'][:ind_fr - 2]

                    
                text = data['text'].split(';')
                if '/' in text[0]:
                    c_orig, c_cor = text[0].split('/')
                    if ',' in c_cor:
                        c_cor = c_cor.split(',')[0]
                else:
                    c_orig, c_cor = ['?', '?']
                constr = text[1].strip() if len(text) > 1 else '?'
                
                author = list(l['author'])[0]
                genre = list(l['genre'])[0]
                gendre = list(l['gender'])[0]
                course = list(l['course'])[0]
                text_type = list(l['text_type'])[0]
                words = int(l['words'])
                sentences = int(l['sentences'])
                
                new_data.append([sentence, errors['quote'][i], errors['corr'][i], gc, ems, ft, level, before,
                                 c_orig.strip(), c_cor.strip(), constr, calque,
                                 author, genre, gendre, course, text_type, 
                                words, sentences])
        except ValueError:
            print(data)
            print(text, '\n')
    
    if new_data != []:
        for i in range(len(new_data)):
            df.loc[i] = new_data[i]
    return df

In [259]:
bilan_fl = pd.DataFrame(columns = ['sentence', 'original', 'corrected', 'Gov/Constr', 'Extra/Miss/Subst', 'Fusion/Transfer/-', 'level',
                                'before', 'constr original', 'constr corrected', 'constr', 'calque',
                                    'author', 'genre', 'gender', 'course', 'text_type', 'words', 'sentences'])
bilan_fl = take_info(bilan_fl, doc_fl, sent_fl, errors_fl)

In [260]:
bilan_hl = pd.DataFrame(columns = ['sentence', 'original', 'corrected', 'Gov/Constr', 'Extra/Miss/Subst', 'Fusion/Transfer/-', 'level',
                                'before', 'constr original', 'constr corrected', 'constr', 'calque',
                                    'author', 'genre', 'gender', 'course', 'text_type', 'words', 'sentences'])
bilan_hl = take_info(bilan_hl, doc_hl, sent_hl, errors_hl)

In [261]:
bilan_fl.head()

,sentence,original,corrected,Gov/Constr,Extra/Miss/Subst,Fusion/Transfer/-,level,before,constr original,constr corrected,constr,calque,author,genre,gender,course,text_type,words,sentences
0,Я буду в июле весь месяц.,в июле весь месяц,"весь июль месяц,",Constr,Extra,Transfer,B1,"c фр. ""j'y serai tout le mois en juillet""",?,?,?,1,Sophia (native bulgare),answers,ж,NaN,NaN,153,18
1,"Официальный язык России - русский, он принадле...",принадлежит группе,принадлежит к группе,Gov,Miss,-,C1,D / к D; Verb,D,к D,Verb,0,Danuta (native slavic),description,ж,NaN,NaN,165,20
2,"В соседнем здании, радио передавало музыку и д...",радио передавало,по радио передавали,Gov,Miss,Transfer,B1,"N / по D, с фр. ""la radio diffusait""",N,по D,?,1,Geraldine T.,other,ж,NaN,NaN,160,12
3,"Москва – 8,4 млн. чел.",Москва,В Москве,Constr,Miss,-,B2,N / в L,N,в L,?,0,Ewelina (native polish),description,ж,L2 Expression écrite et géographie,NaN,234,23
4,"Волки гнают муха, они наблюдают медведь.",гнают муха,"гонятся за мухой,",Gov,Miss,-,A2,N / за I; Verb,N,за I,Verb,0,Anaïs F.,description,ж,L3 Grammaire/Traduction,NaN,81,13


In [262]:
# сохраняем в xlsx

xlWriter = pd.ExcelWriter('constructions_2.xlsx', engine='xlsxwriter')
workbook = xlWriter.book
bilan_fl.to_excel(xlWriter, sheet_name='fl')
bilan_hl.to_excel(xlWriter, sheet_name='hl')
xlWriter.save()